In [4]:
import numpy as np
import deepchem as dc
from source.utils.file_handling_utils import ls, silentremove
from source.utils.npz_utils import get_field_from_npzs

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/nobilm@usi.ch/miniconda3/envs/rdkit_pyg_geqm9_clone/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [5]:
# TODO:

# mp func to
# given dir
# get as out:
# 1) do npz-check, -> rm if needed
# 2) {filepath: ..., npz_content: ...}

def f(dir):
    files = ls(dir)
    out = []
    for f in files:
        if not test_npz(f):
            continue
        npz = get_field_from_npzs(f)
        out.append({
            'filename': npz[0].zip.filename,
            'smiles': npz[0]['smiles']
        })
    return out

def test_npz(file):
    try:
        np.load(file, allow_pickle=True)
        return True
    except Exception as e:
        print(f'Error loading {file}: {e}. Removed {file}')
        silentremove(file)
        return False

# update smile_list to be out of above
def scaffold_splitter(d, data_dir, frac_train: float = 0.8, frac_valid: float = 0.1, frac_test: float = 0.1, seed: int=42):
    # data_dir (str, optional (default None)) – The directory to write this dataset to. If none is specified, will use a temporary directory instead.
    '''https://deepchem.readthedocs.io/en/latest/api_reference/splitters.html#scaffoldsplitter
    https://deepchem.readthedocs.io/en/latest/api_reference/data.html#deepchem.data.DiskDataset.from_numpy
    '''

    if data_dir == 'tmp':
        data_dir = None

    smile_list = [str(el['smiles']) for el in d]
    filepath = [str(el['filename']) for el in d]
    assert len(smile_list) == len(filepath)
    fake_x = np.zeros(len(smile_list))
    fake_y = np.ones(len(smile_list))
    # creation of a deepchem dataset with the smile codes in the ids field
    dataset = dc.data.DiskDataset.from_numpy(X=fake_x,y=fake_y,w=np.zeros(len(smile_list)),ids=smile_list, data_dir=data_dir)
    scaffoldsplitter = dc.splits.ScaffoldSplitter()
    train_idxs, val_idxs, test_idxs = scaffoldsplitter.split(dataset, frac_train, frac_valid, frac_test, seed)

    train_smiles = [{'filepath':dataset.X[index], 'smiles':smile_list[index]} for index in train_idxs]
    val_smiles = [{'filepath':dataset.X[index], 'smiles':smile_list[index]} for index in val_idxs]
    test_smiles = [{'filepath':dataset.X[index], 'smiles':smile_list[index]} for index in test_idxs]

    return train_smiles,val_smiles,test_smiles


In [6]:
dir = '/storage_common/nobilm/pretrain_paper/guacamol/5k'
d = f(dir)

In [7]:
train_smiles,val_smiles,test_smiles=scaffold_splitter(d, 'tmp')

In [8]:
train_smiles

[{'filepath': 0.0, 'smiles': 'C=C(C#N)C(OC(C)=O)c1ccc([N+](=O)[O-])cc1'},
 {'filepath': 0.0, 'smiles': 'CC(=O)Nc1ccc(S(=O)(=O)N=S(C)C)cc1'},
 {'filepath': 0.0, 'smiles': 'N#CC(C#N)=Cc1ccc(O)cc1'},
 {'filepath': 0.0, 'smiles': 'CC(C)CNC(=O)NC(=O)COc1ccc(C=O)cc1'},
 {'filepath': 0.0, 'smiles': 'CCCCCCCCCCC(=O)NCCc1ccc(OC)c(OC)c1'},
 {'filepath': 0.0, 'smiles': 'C=CCc1cc(Cl)ccc1OCCOCCNCCOC'},
 {'filepath': 0.0, 'smiles': 'CCCCCC=CCC=CCCCCCCCC(=O)Nc1c(OC)cc(OC)cc1OC'},
 {'filepath': 0.0, 'smiles': 'CCCCCCCCCCCCCCCc1ccc(O)cc1O'},
 {'filepath': 0.0, 'smiles': 'CN(Cc1ccc(Cl)c(Cl)c1)C(=O)CNC(N)=O'},
 {'filepath': 0.0, 'smiles': 'NC(Cc1ccccc1)C(=O)NC(CC(=O)O)C(=O)O'},
 {'filepath': 0.0, 'smiles': 'O=C(O)CCCCCONC(=O)Nc1ccccc1'},
 {'filepath': 0.0, 'smiles': 'CCC(C)Oc1cc(NC(C)=O)ccc1C(=O)OC'},
 {'filepath': 0.0, 'smiles': 'CCCCCCCCC(C)(C)C(=O)Nc1c(OC)cc(OC)cc1OC'},
 {'filepath': 0.0, 'smiles': 'O=C(O)CC(NC(=O)C(F)(F)F)C(=O)Nc1ccccc1'},
 {'filepath': 0.0, 'smiles': 'O=C(O)c1ccc(N(CCBr)CCBr)cc1'},
